In [2]:
"""
    20250108从GTA数据库中下载可转债信息表、可转债和正股日线信息表
"""
import pandas as pd
import numpy as np
import time
import os
import utils
import pyodbc

In [ ]:
def getBond_ST_New(config, BondInfo):
    df_all = []
    for nS in range(0, len(BondInfo)):
        codeCB = BondInfo['SYMBOL9'][nS]
        codeSTK = BondInfo['OBJECTSTOCKCODE9'][nS]
        if nS%500 == 1:
            print('getBond_ST_New, nS = ', str(nS), codeCB, codeSTK)
        #
        df_CB = []
        file_path = os.path.join(config['Info_dir'], 'Bond/', codeCB + '.csv')
        if os.path.exists(file_path):
            df_CB = pd.read_csv(file_path)
            df_CB = df_CB.dropna(subset=['CLOSEPRICE'])
            df_CB = df_CB[df_CB['AMOUNT'] > 0]
        if len(df_CB) == 0:
            # print(config['Info_dir'], 'Bond数据为空，', codeCB)
            continue
        #
        df_STK = []
        file_path_STK = os.path.join(config['Info_dir'], 'STK_Fwd', codeSTK + '.csv')
        if os.path.exists(file_path_STK):
            df_STK = pd.read_csv(file_path_STK)
            df_STK = df_STK.dropna(subset=['CLOSEPRICE'])
            df_STK = df_STK[df_STK['AMOUNT'] > 0]
            df_STK = df_STK[['TRADINGDATE', 'isST']]
        #
        df = df_CB[['TRADINGDATE', 'SYMBOL9']].copy().reset_index()
        df['Fund_NewBnd'] = 0
        if len(df) <= 20:
            df['Fund_NewBnd'] = 1
        else:
            if df['TRADINGDATE'].iloc[0] > '2022-01-04':
                df.loc[df.index[:20], 'Fund_NewBnd'] = 1
        #
        df['Fund_FlagST'] = False
        if len(df_STK) > 0:
            df = df.merge(df_STK, on='TRADINGDATE', how='left', sort=False)
            df.loc[df['isST'], 'Fund_FlagST'] = True
        # 
        if len(df_all) == 0:
            df_all = df
        else:
            df_all = pd.concat([df_all, df], ignore_index=True)
    #
    count = df_all.groupby('TRADINGDATE')['SYMBOL9'].count().values
    print('all count = ', count[-100:])

    df_all2 = df_all[df_all['Fund_FlagST']]
    if len(df_all)>0:
        print(df_all2)

    #
    factorlist = ['Fund_NewBnd', 'Fund_FlagST']
    for factorname in factorlist:
        df2 = df_all[['SYMBOL9', 'TRADINGDATE', factorname]].copy()
        df2.rename(columns={factorname: 'FACTORVALUE'}, inplace=True)
        file_path = os.path.join(config['factor_dir'], 'Array/' + factorname + '.csv')
        df2.to_csv(file_path, index=False)
        #
        file_path = os.path.join(config['factor_dir'], 'Array/' + factorname + '.parquet')
        df2.to_parquet(file_path, engine='pyarrow')
        #
        if factorname == 'Fund_NewBnd':
            df2 = df_all[df_all[factorname] == 1]
        elif factorname == 'Fund_FlagST':
            df2 = df_all[df_all[factorname]]
        count = df2.groupby('TRADINGDATE')['SYMBOL9'].count().values
        print(factorname, ' count = ', count)
        
    return 0

# 下载交易日历（每天是否交易 + 年/月字段）
def quote_CALENDARD(config):
    conn_str = (
        r'DRIVER={SQL Server};'
        r'SERVER=' + config['IP_QDB'] + ';'
        r'DATABASE=GTA_QIA_QDB;'
        r'UID=sa;'
        r'PWD=Tantra612;'
    )
    conn = pyodbc.connect(conn_str)
    # 创建游标对象
    cursor = conn.cursor()
    # 执行SQL查询
    cursor.execute("SELECT * FROM STK_CALENDARD")
    rows = cursor.fetchall()
    rows = pd.DataFrame.from_records(rows)
    # 查询表头
    cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'STK_CALENDARD'")
    tempTitle = cursor.fetchall()
    tempTitle = pd.DataFrame.from_records(tempTitle)
    tempTitle = tempTitle[0]

    rows.columns = tempTitle
    rows = rows.sort_values(by=['CALENDARDATE', 'EXCHANGECODE'], ascending=True)

    df = rows[rows['CALENDARDATE'] >= '2010-01-01']
    df = df.drop(['UPDATEID', 'UPDATESTATE', 'UPDATETIME', 'BUSINESSTIME', 'UTSID'], axis=1)
    df = df.reset_index(drop=True)
    # df = df.drop(['index'], axis=1)

    # df['TradeDate'] = df['CALENDARDATE'].apply(
    #     lambda x: int(datetime.strptime(x, '%Y-%m-%d').strftime('%Y%m%d')))
    df['TradeDate'] = df['CALENDARDATE'].apply(lambda x: int(x.strftime('%Y%m%d')))
    df['TradeMth'] = df['CALENDARDATE'].apply(lambda x: int(x.strftime('%Y%m')))
    df['TradeYear'] = df['CALENDARDATE'].apply(lambda x: int(x.strftime('%Y')))

    # 关闭游标和连接
    cursor.close()
    conn.close()
    return df

# 下载可转债信息表
def quote_BOND_ConvertInfo(config):
    conn_str = (
        r'DRIVER={SQL Server};'
        r'SERVER=' + config['IP_QDB'] + ';'
        r'DATABASE=GTA_QIA_QDB;'
        r'UID=sa;'
        r'PWD=Tantra612;'
    )
    conn = pyodbc.connect(conn_str)
    # 创建游标对象
    cursor = conn.cursor()
    # 执行SQL查询
    cursor.execute("SELECT * FROM BOND_ConvertInfo where MaturityDate >= '2020-01-01'")
    rows = cursor.fetchall()
    # 遍历并打印结果
    # for row in rows:
    #     print(row)
    rows = pd.DataFrame.from_records(rows)
    # TempTitle = {'BondID', '债券主体ID';
    # 'SecurityID', '证券ID';
    # 'Symbol', '债券代码';
    # 'ShortName', '债券简称';
    # 'ExchangeCode', '交易市场编码';
    # 'Bondyear', '债券年度';
    # 'BondTypeID', '品种类别ID';
    # 'BondType', '品种类别';
    # 'ObjectStockCode', '可转换股票代码';
    # 'IssueQuantity', '实际发行量';
    # 'IssuePrice', '发行价格';
    # 'BondTerm', '期限';
    # 'ListDate', '上市日期';
    # 'IssueDate', '发行日期';
    # 'MaturityDate', '到期日期';
    # 'StartDate', '计息日期';
    # 'EndDate', '止息日期';
    # 'InterestPayType', '付息方式';
    # 'Interest', '利率类型';
    # 'InterestType', '计息方式';
    # 'PayInterestFrequency', '年付息次数';
    # 'PayInterestDate', '付息日期描述';
    # 'InterestRate', '票面利率';
    # 'BenchmarkRate', '基准利率';
    # 'BaseYieldSpread', '基本利差';
    # 'ParValue', '面值';
    # 'CreditRate', '信用等级';
    # 'ConvertStartDate', '转股起始日期';
    # 'ConvertEndDate', '转股截止日期';
    # 'ConvertStartPrice', '初始转股价格';
    # 'IsCall', '可赎回性';
    # 'CallStartDate', '可赎回起始日期';
    # 'CallEndDate', '可赎回截止日期';
    # 'CallPrice', '赎回价格';
    # 'IsPut', '可回售性';
    # 'PutStartDate', '回售起始日期';
    # 'PutEndDate', '回售截止日期';
    # 'PutPrice', '回售价格'};

    # 查询表头
    cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'BOND_ConvertInfo'")
    tempTitle = cursor.fetchall()
    tempTitle = pd.DataFrame.from_records(tempTitle)
    tempTitle = tempTitle[0]
    rows.columns = tempTitle
    #
    rows.loc[rows['OBJECTSTOCKCODE'].str[0] == '0', 'OBJECTSTOCKCODE2'] = 'SZ'
    rows.loc[rows['OBJECTSTOCKCODE'].str[0] == '3', 'OBJECTSTOCKCODE2'] = 'SZ'
    rows.loc[rows['OBJECTSTOCKCODE'].str[0] == '6', 'OBJECTSTOCKCODE2'] = 'SH'
    rows['OBJECTSTOCKCODE9'] = rows['OBJECTSTOCKCODE'].astype(str) + '.' + rows['OBJECTSTOCKCODE2']
    df = rows[pd.isna(rows['OBJECTSTOCKCODE9'])]
    rows = rows[~rows['OBJECTSTOCKCODE9'].isna()]
    #
    rows['SYMBOL9'] = rows['SYMBOL'].astype(str) + '.' + rows['OBJECTSTOCKCODE2']
    rows = rows.drop(['OBJECTSTOCKCODE2'], axis=1)
    rows = rows.sort_values(by=['SYMBOL'], ascending=True)
    rows = rows.reset_index()

    # 关闭游标和连接
    cursor.close()
    conn.close()
    return rows

# 下载可转债个债信息表
def quote_BOND_Quotation_Mth(config, STKlist, TradeDays):
    conn_str = (
        r'DRIVER={SQL Server};'
        r'SERVER=' + config['IP_QDB'] + ';'
        r'DATABASE=GTA_QIA_QDB;'
        r'UID=sa;'
        r'PWD=Tantra612;'
    )
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    # 查询表头
    cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'BOND_Quotation'")
    tempTitle = cursor.fetchall()
    tempTitle = pd.DataFrame.from_records(tempTitle)
    tempTitle = tempTitle[0]

    # TradeDays = utils.fetchCalendard2Mth(TradeDays)
    TradeDaysG = TradeDays.groupby('TradeMth')

    folder_path = config['Info_dir'] + config['folder_path'] + '/Mth/'
    os.makedirs(folder_path, exist_ok=True)
    df_all = []
    for name, group in TradeDaysG:
        name = str(name)
        # print(f"Group name: {name}")
        # print(group)
        start_time = group['CALENDARDATE'].iloc[0]
        end_time = group['CALENDARDATE'].iloc[-1]
        print(end_time)
        #
        filename = f"BOND_Quotation_{name}.csv"
        file_path = os.path.join(folder_path, filename)
        if name < '202502' and os.path.exists(file_path):
            continue
        else:
            print(name)
        
        Sql_Str=f"SELECT * FROM BOND_Quotation where BondNatureID='S0305' and TRADINGDATE >='{start_time}' and TRADINGDATE<='{end_time}'"
        cursor.execute(Sql_Str)
        rows = cursor.fetchall()
        # 遍历并打印结果
        rows = pd.DataFrame.from_records(rows)
        if len(rows) == 0:
            print('rows为空')
            continue
        rows.columns = tempTitle
        #
        exchange_dict = {"SSE": "SH", "SZSE": "SZ"}
        rows['EXCHANGECODE'] = rows['EXCHANGECODE'].map(exchange_dict)
        rows['SYMBOL9'] = rows['SYMBOL'].astype(str) + '.' + rows['EXCHANGECODE']
        #
        rows = rows.sort_values(by=['SYMBOL', 'TRADINGDATE'], ascending=True)
        rows = rows.drop(['BONDNATUREID', 'UPDATEID', 'UPDATESTATE', 'UPDATETIME', 'BUSINESSTIME', 'UTSID'], axis=1)
        rows.to_csv(file_path, index=False)

    # TempTitle_sav={'TradingDate','交易日期';'OpenPrice','开盘价';'HighPrice','最高价';'LowPrice','最低价';'ClosePrice','收盘价';...
    #     'Volume','成交量';'Amount','成交额';'TurnoverRate','换手率';'Change','涨跌';'ChangeRatio','涨跌幅';'Amplitude','振幅';...
    #     'InterestDays','计息天数';'AccruedInterest','应计利息';'GrossPrice','收盘全价';'RemainingYears','剩余年限';'StockPrice','正股价格';...
    #     'StockChangeRatio','正股涨跌幅';'ConversionPrice','转股价格';'ConversionRatio','转股比例';'ConversionValue','转换价值';...
    #     'ConvertiblePremiumRate','转股溢价率';'ChangetoIssuePrice','相对发行价涨跌';'ChangeRatiotoIssue','相对发行价涨跌幅';'AvgPrice','均价';...
    #     'Filling','填充标识'};

    # 关闭游标和连接
    cursor.close()
    conn.close()
    return rows

# 下载正股个股信息表--按月抽取正股的行情
def quote_STK_MKT_Quotation_Mth(config, STKlist, TradeDays):
    conn_str = (
        r'DRIVER={SQL Server};'
        r'SERVER=' + config['IP_QDB'] + ';'
        r'DATABASE=GTA_QIA_QDB;'
        r'UID=sa;'
        r'PWD=Tantra612;'
    )
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    # 查询表头
    if '_Fwd' in config['folder_path']:
        TABLE_NAME = 'STK_MKT_FwardQuotation'
    else:
        TABLE_NAME = 'STK_MKT_Quotation'
    Sql_Str = f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME ='{TABLE_NAME}'"
    # cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'STK_MKT_FwardQuotation'")
    cursor.execute(Sql_Str)
    tempTitle = cursor.fetchall()
    tempTitle = pd.DataFrame.from_records(tempTitle)
    tempTitle = tempTitle[0]

    # TradeDays = utils.fetchCalendard2Mth(TradeDays)
    TradeDaysG = TradeDays.groupby('TradeMth')

    folder_path = config['Info_dir'] + config['folder_path'] + '/Mth/'
    os.makedirs(folder_path, exist_ok=True)
    for name, group in TradeDaysG:
        name = str(name)
        filename = f"{TABLE_NAME}_{name}.csv"
        file_path = os.path.join(folder_path, filename)
        if name < '202502' and os.path.exists(file_path):
            continue
        else:
            print(TABLE_NAME, ' ', name)
        #
        print(f"Group name: {name}")
        # print(group)
        start_time = group['CALENDARDATE'].iloc[0]
        end_time = group['CALENDARDATE'].iloc[-1]
        print(end_time)
        #
        Sql_Str=f"SELECT * FROM {TABLE_NAME} where TRADINGDATE >='{start_time}' and TRADINGDATE<='{end_time}'"
        cursor.execute(Sql_Str)
        rows = cursor.fetchall()
        rows = pd.DataFrame.from_records(rows)
        if len(rows) == 0:
            print('rows为空')
            continue
        rows.columns = tempTitle
        if TABLE_NAME == 'STK_MKT_FwardQuotation':
            rows = rows.drop(['UPDATEID', 'UPDATESTATE', 'UPDATETIME', 'BUSINESSTIME', 'UTSID'], axis=1)
        else:
            rows = rows.drop(['UPDATEID', 'UPDATESTATE', 'UPDATETIME', 'BUSINESSTIME', 'UTSID', 'AFTERHOURSVOLUME', 'AFTERHOURSAMOUNT'], axis=1)
        rows = rows.sort_values(by=['SYMBOL', 'TRADINGDATE'], ascending=True)
        #
        STKlist2 = [x[:6] for x in STKlist]
        # rows = utils.dropSTK_not_inTargetPool(rows, STKlist2)
        rows = rows[rows['SYMBOL'].isin(STKlist2)]
        # 增加ST标记，和股票代码转换为600000.SH
        rows['isST'] = rows['SHORTNAME'].str.contains('ST')
        # 转9位代码
        rows = utils.symbol6_to_symbol9(rows)

        rows.to_csv(file_path, index=False)

    # 关闭游标和连接
    cursor.close()
    conn.close()
    return 0

# 下载正股行业信息表
def quote_STK_INDUSTRYCLASS(config):
    conn_str = (
        r'DRIVER={SQL Server};'
        r'SERVER=' + config['IP_QDB'] + ';'
        r'DATABASE=GTA_QIA_QDB;'
        r'UID=sa;'
        r'PWD=Tantra612;'
    )
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    # 查询行业分类标准
    cursor.execute("SELECT * FROM PUB_INDCLASSIFYSETS where INDCLASSIFYSYSTEMCODE = 'P0218'")
    rows = cursor.fetchall()
    # 遍历并打印结果
    # for row in rows:
    #     print(row)
    classinfo = pd.DataFrame.from_records(rows)
    # 查询表头
    cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'PUB_INDCLASSIFYSETS'")
    tempTitle = cursor.fetchall()
    tempTitle = pd.DataFrame.from_records(tempTitle)
    tempTitle = tempTitle[0]
    classinfo.columns = tempTitle
    classinfo = classinfo.drop(['UPDATEID', 'UPDATESTATE', 'UPDATETIME', 'BUSINESSTIME', 'UTSID'], axis=1)

    # 查询个股行业标记
    cursor.execute("SELECT * FROM STK_INDUSTRYCLASS where INDUSTRYCLASSIFICATIONID = 'P0218'")
    rows = cursor.fetchall()
    classdata = pd.DataFrame.from_records(rows)
    # 查询表头
    cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'STK_INDUSTRYCLASS'")
    tempTitle = cursor.fetchall()
    tempTitle = pd.DataFrame.from_records(tempTitle)
    tempTitle = tempTitle[0]
    classdata.columns = tempTitle
    classdata = utils.symbol6_to_symbol9(classdata)
    classdata = classdata[~classdata['SYMBOL9'].isna()]
    #
    classdata = classdata.drop(['UPDATEID', 'UPDATESTATE', 'UPDATETIME', 'BUSINESSTIME', 'UTSID'], axis=1)
    classdata = classdata.sort_values(by=['CHANGEDATE'], ascending=False)
    classdata = classdata.drop_duplicates(subset='SYMBOL9', keep='first')
    classdata = classdata.sort_values(by=['SYMBOL'], ascending=True)
    classdata = classdata.reset_index()
    # 匹配一二三级行业代码和行业名称
    classdata[['INDUSTRYCODE3', 'INDUSTRYNAME3', 'INDUSTRYCODE2', 'INDUSTRYNAME2', 'INDUSTRYCODE1', 'INDUSTRYNAME1']] = np.nan
    tempinfo = classinfo[classinfo['RANK'] == 3]
    for nC in range(0, len(tempinfo)):
        code = tempinfo['INDUSTRYCODE'].iloc[nC]
        name = tempinfo['INDUSTRYNAME'].iloc[nC]
        # print(code, name)
        classdata.loc[classdata['INDUSTRYCODE'] == code, 'INDUSTRYCODE3'] = code
        classdata.loc[classdata['INDUSTRYCODE'] == code, 'INDUSTRYNAME3'] = name
        classdata.loc[classdata['INDUSTRYCODE'] == code, 'INDUSTRYCODE2'] = code[:4] + '00'
        classdata.loc[classdata['INDUSTRYCODE'] == code, 'INDUSTRYCODE1'] = code[:2] + '0000'
    #
    tempinfo = classinfo[classinfo['RANK'] == 2]
    for nC in range(0, len(tempinfo)):
        code = tempinfo['INDUSTRYCODE'].iloc[nC]
        name = tempinfo['INDUSTRYNAME'].iloc[nC]
        classdata.loc[classdata['INDUSTRYCODE2'] == code, 'INDUSTRYNAME2'] = name
    #
    tempinfo = classinfo[classinfo['RANK'] == 1]
    for nC in range(0, len(tempinfo)):
        code = tempinfo['INDUSTRYCODE'].iloc[nC]
        name = tempinfo['INDUSTRYNAME'].iloc[nC]
        classdata.loc[classdata['INDUSTRYCODE1'] == code, 'INDUSTRYNAME1'] = name
    # print(len(classdata['INDUSTRYNAME1'].unique()))

    # 关闭游标和连接
    cursor.close()
    conn.close()
    return classdata

In [7]:
if __name__ == '__main__':
    YYYYMMDD = time.strftime("%Y%m%d", time.localtime())
    HHMMSS = time.strftime("%H%M%S", time.localtime())
    print(YYYYMMDD, HHMMSS)
    tempFlag = 0
    # while tempFlag == 0:
    #     YYYYMMDD = time.strftime("%Y%m%d", time.localtime())
    #     HHMMSS = time.strftime("%H%M%S", time.localtime())
    #     print(YYYYMMDD, HHMMSS)
    #     if (HHMMSS > '230003') or (HHMMSS < '90000'):
    #         tempFlag = 9999
    #     else:
    #         time.sleep(119)

    config = {
        'IP_QDB': '192.168.0.131',
        'Info_dir': 'D:/data/STKInfo/',
        'HQ_dir': 'D:/data/STKBar/',
        'factor_dir': 'D:/data/STKInfo/',
        'start_time': '20220101',
        'end_time': '20250707',
        'adj_type': 'f'
    }
    today = time.strftime("%Y-%m-%d", time.localtime())

    # 更新日历表
    df = quote_CALENDARD(config)
    df.to_csv(config['Info_dir'] + 'STK_CALENDARD.csv', index = False)
    # df = pd.read_csv(config['Info_dir'] + 'STK_CALENDARD.csv', dtype=str, parse_dates=['CALENDARDATE'])
    df = pd.read_csv(config['Info_dir'] + 'STK_CALENDARD.csv', parse_dates=['CALENDARDATE'])
    df = df[(df['CALENDARDATE'] > '2022-01-01') & (df['CALENDARDATE'] <= today)]
    df = df[df['ISOPEN'] == 'Y']
    df = df[(df['EXCHANGECODE'] == 'SSE') | (df['EXCHANGECODE'] == 'SZSE')]
    TradeDays = df
    # print(df)
    print('更新日历表结束')

    # 下载可转债信息表
    df = quote_BOND_ConvertInfo(config)
    df.to_csv(config['Info_dir'] + 'BOND_ConvertInfo.csv', index = False)
    df = pd.read_csv(config['Info_dir'] + 'BOND_ConvertInfo.csv', dtype=str)
    exchange_dict = {
        "SSE": "SH",
        "SZSE": "SZ"
    }
    df['EXCHANGECODE'] = df['EXCHANGECODE'].map(exchange_dict)
    # 提取可转债代码
    df['tr_CBcode'] = df['SYMBOL'].astype(str) + '.' + df['EXCHANGECODE']
    all_CB_list = df['tr_CBcode'].tolist()
    # 提取正股代码
    df['tr_STKcode'] = df['OBJECTSTOCKCODE'].astype(str) + '.' + df['EXCHANGECODE']
    all_STK_list = df['tr_STKcode'].tolist()
    # print(df)
    print('更新转债信息表结束')

    # # 下载正股行业信息表
    # df = quote_STK_INDUSTRYCLASS(config)
    # df.to_csv(config['Info_dir'] + 'STK_INDUSTRYCLASS.csv', index = False)
    # # print(df)
    # df = pd.read_csv(config['Info_dir'] + 'STK_INDUSTRYCLASS.csv')


    # ##########################################################
    # # 可转债日线Info
    config['end_time'] = TradeDays['TradeDate'].iloc[-1]
    config['folder_path'] = 'Bond'
    empty_code = quote_BOND_Quotation_Mth(config, all_CB_list, TradeDays)
    print('下载可转债日线Info结束')
    print(empty_code)
    print(time.strftime("%Y%m%d %H%M%S", time.localtime()))
    empty_code = utils.splitQuoteInfo2stk(config, all_CB_list, TradeDays)
    print(empty_code)
    print(len(empty_code))
    print('拆分可转债日线Info结束')


    # 正股日线Info
    config['end_time'] = TradeDays['TradeDate'].iloc[-1]
    config['folder_path'] = 'STK_Org'
    empty_code = quote_STK_MKT_Quotation_Mth(config, all_STK_list, TradeDays)
    print('下载正股日线Info结束')
    print(empty_code)
    empty_code = utils.splitQuoteInfo2stk(config, all_STK_list, TradeDays)
    print(empty_code)
    print(len(empty_code))
    print('拆分正股日线Info结束')


    # # 正股日线FwdInfo
    config['end_time'] = TradeDays['TradeDate'].iloc[-1]
    config['folder_path'] = 'STK_Fwd'
    empty_code = quote_STK_MKT_Quotation_Mth(config, all_STK_list, TradeDays)
    print('下载正股日线FwdInfo结束')
    print(empty_code)
    empty_code = utils.splitQuoteInfo2stk(config, all_STK_list, TradeDays)
    print('拆分正股日线FwdInfo结束')
    print(empty_code)
    print(len(empty_code))


    # 生成可转债的新债和ST因子
    BondInfo = pd.read_csv(config['Info_dir'] + 'BOND_ConvertInfo.csv')
    df = getBond_ST_New(config, BondInfo)

20251030 141119
更新日历表结束
更新转债信息表结束
2022-01-28 00:00:00
2022-02-28 00:00:00
2022-03-31 00:00:00
2022-04-29 00:00:00
2022-05-31 00:00:00
2022-06-30 00:00:00
2022-07-29 00:00:00
2022-08-31 00:00:00
2022-09-30 00:00:00
2022-10-31 00:00:00
2022-11-30 00:00:00
2022-12-30 00:00:00
2023-01-31 00:00:00
2023-02-28 00:00:00
2023-03-31 00:00:00
2023-04-28 00:00:00
2023-05-31 00:00:00
2023-06-30 00:00:00
2023-07-31 00:00:00
2023-08-31 00:00:00
2023-09-28 00:00:00
2023-10-31 00:00:00
2023-11-30 00:00:00
2023-12-29 00:00:00
2024-01-31 00:00:00
2024-02-29 00:00:00
2024-03-29 00:00:00
2024-04-30 00:00:00
2024-05-31 00:00:00
2024-06-28 00:00:00
2024-07-31 00:00:00
2024-08-30 00:00:00
2024-09-30 00:00:00
2024-10-31 00:00:00
2024-11-29 00:00:00
2024-12-31 00:00:00
2025-01-27 00:00:00
2025-02-28 00:00:00
202502
2025-03-31 00:00:00
202503
2025-04-30 00:00:00
202504
2025-05-30 00:00:00
202505
2025-06-30 00:00:00
202506
2025-07-31 00:00:00
202507
2025-08-29 00:00:00
202508
2025-09-30 00:00:00
202509
2025-10-30